In [1]:
import numpy as np
import os
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [44]:
train = pd.read_csv('./data/round1_iflyad_train.txt', sep='\t')
test = pd.read_csv('./data/round1_iflyad_test_feature.txt', sep='\t')

train.shape
train.head()
test.shape
test.head()

(1001650, 35)

,instance_id,time,city,province,user_tags,carrier,devtype,make,model,nnt,os,osv,os_name,adid,advert_id,orderid,advert_industry_inner,campaign_id,creative_id,creative_tp_dnf,app_cate_id,f_channel,app_id,inner_slot_id,creative_type,creative_width,creative_height,creative_is_jump,creative_is_download,creative_is_js,creative_is_voicead,creative_has_deeplink,app_paid,advert_name,click
0,86294719979897807,2190219034,137103102105100,137103102100100,NaN,1,2,HUAWEI,HUAWEI-CAZ-AL10,1,2,7.0.0,android,1560128,230000063,3010798,102400_102401,1000023,2338420,8390267734059046014,107.0,NaN,2089229.0,xf_275C061483984E075832A4373BDDF27B,8,1280,720,True,False,False,False,False,False,B4734117F35EE97F,0
1,2699289844928136052,2190221070,137105101100100,137105101100100,"2100191,2100078,3001825,,3001781,3001791,30017...",3,2,Xiaomi,Redmi Note 4,1,2,6.0,android,1488859,230000063,2311397,102400_102401,1000023,2310417,8390208550469153745,108.0,NaN,2070079.0,xf_D84DAB691E2E08C5B80D2FF5135F886E,8,960,640,True,False,False,False,False,False,B4734117F35EE97F,0
2,3117527168445845752,2190219793,137103104111100,137103104100100,NaN,3,2,OPPO,OPPO+R11s,1,2,7.1.1,android,1537089,230000065,3008491,101700_101704,1000021,2337017,8390430283595430291,100.0,NaN,2089397.0,xf_7F9FF3BEA11FE5B3AE6332EFBBD59496,8,960,640,True,False,False,False,False,False,E257895F74792E81,0
3,3398484891050993371,2190221704,137103102113100,137103102100100,"2100098,gd_2100000,3001791,3001795,3002193,300...",0,2,NaN,OPPO A57,1,2,6.0.1,android,1577884,230001710,3011304,101900_101902,1003544,2342152,8390229093704413749,100.0,iqy_2,2071234.0,iqy_1000000000381-1-15-15,3,1280,720,True,False,False,False,False,False,0A421D7B11EABFC5,0
4,2035477570591176488,2190220024,137103102109100,137103102100100,NaN,1,2,Apple,iPhone 7,3,1,11.1.1,ios,1432367,230000063,2304491,102400_102401,1000023,2305409,8390208550469153745,103.0,NaN,1030051.0,xf_6C4DCB36DBE7EB12CE55EDF319FF8D93,8,960,640,True,False,False,False,False,False,B4734117F35EE97F,0


(40024, 34)

,instance_id,time,city,province,user_tags,carrier,devtype,make,model,nnt,os,osv,os_name,adid,advert_id,orderid,advert_industry_inner,campaign_id,creative_id,creative_tp_dnf,app_cate_id,f_channel,app_id,inner_slot_id,creative_type,creative_width,creative_height,creative_is_jump,creative_is_download,creative_is_js,creative_is_voicead,creative_has_deeplink,app_paid,advert_name
0,6930856710792380886,2190675456,137103104101100,137103104100100,NaN,2,2,Apple,iPhone 8 Plus,1,1,11.4.1,ios,1518579,230000063,3006011,102400_102401,1000023,2316955,8390430283595430291,103.0,NaN,1030051.0,xf_6C4DCB36DBE7EB12CE55EDF319FF8D93,8,960,640,True,False,False,False,False,False,B4734117F35EE97F
1,5460409694420131920,2190674821,137103104112100,137103104100100,"3004406,3004430,3004434",1,2,vivo,vivo X9Plus,1,2,7.1.2,android,1580740,230000063,3005949,102400_102401,1000023,2334795,8390430283595430291,108.0,NaN,1080001.0,xf_826A5605FD0EBC210098D47779C21CBE,8,960,640,True,False,False,False,False,False,B4734117F35EE97F
2,982813438159141507,2190674111,137105103101100,137105103100100,"3003779,3003843,3003851,3003863,3003865,300386...",2,2,OPPO A73t,OPPO A73t,4,2,7.1.1,android,1585472,230000063,3012756,102400_102401,1000023,2313211,8390465404427801098,111.0,NaN,1110016.0,xf_EE10823F5C5D57FF941653A9B5BA7B07,5,160,640,True,False,False,False,False,False,B4734117F35EE97F
3,529991959116679673,2190675256,137106101107100,137106101100100,NaN,2,2,vivo Z1,vivo Z1,4,2,8.1.0,android,1575100,230000063,2311393,102400_102401,1000023,2334795,8390430283595430291,111.0,NaN,1110016.0,xf_9EFA0BFCFEDCE6FF7B644299ACB88167,8,960,640,True,False,False,False,False,False,B4734117F35EE97F
4,5357053206615171780,2190673926,137103102101100,137103102100100,"2100191,2100041,2100078,2100136,2100042,300182...",3,2,HUAWEI,HUAWEI MLA-AL10,4,2,6.0,android,1584212,230000191,3012566,101700_101704,1003348,2338134,8390272894417697957,107.0,NaN,1000049.0,xf_64FAC0085FC8DC679F6BF49A1F98445D,5,320,480,True,False,False,False,False,False,42A4CB9035B7F50E


In [127]:
test['click'] = -1
df = pd.concat([train, test], axis=0)
df.reset_index(drop=True, inplace=True)

df.shape

(1041674, 35)

In [128]:
df.click.value_counts()

 0    802863
 1    198787
-1     40024
Name: click, dtype: int64

## 缺失值

In [129]:
df.isnull().sum()[df.isnull().sum() != 0]

user_tags      323002
make           103043
model            7793
osv              8211
app_cate_id      2298
f_channel      961897
app_id           2298
dtype: int64

In [130]:
# f_channel太多直接删除
df = df.drop(['f_channel'], axis=1)

In [131]:
# make、model、osv填充固定值表缺失
df.make = df.make.fillna('no_make')
df.model = df.model.fillna('no_model')
df.osv = df.osv.fillna('no_osv')

In [132]:
# app_cate_id、app_id
df.app_cate_id = df.app_cate_id.fillna(-1)
df.app_id = df.app_id.fillna(-1)

## 非int型处理

In [133]:
from collections import Counter, defaultdict

In [134]:
def user_tags_split(x):
    return [] if type(x)==float else x.split(',')
    
def advert_industry_inner_split(x):
    return x.split('_')

In [135]:
df.user_tags = df.user_tags.apply(user_tags_split)
df.advert_industry_inner = df.advert_industry_inner.apply(advert_industry_inner_split)

df['advert_industry_inner_split_0'] = df.advert_industry_inner.apply(lambda x:x[0])
df['advert_industry_inner_split_1'] = df.advert_industry_inner.apply(lambda x:x[1])

df = df.drop(['advert_industry_inner'], axis=1)

In [136]:
user_tags_dict = defaultdict(lambda : 0)

for user_tags in tqdm_notebook(df.user_tags):
    for user_tag in user_tags:
        user_tags_dict[user_tag] += 1
user_tags_most = [i[0] for i in sorted(dict(user_tags_dict).items(), key=lambda x:x[1], reverse=True)[:39]]
user_tags_most.remove('')
len(user_tags_most)

38

In [137]:
def user_tags_list(x):
    tmp = [0] * 38
    for i in x:
        if i in user_tags_most:
            tmp[user_tags_most.index(i)] = 1
    return tmp

df.user_tags = df.user_tags.apply(user_tags_list)

for i in tqdm_notebook(range(38)):
    df['user_tags_' + str(i)] = df.user_tags.apply(lambda x:x.pop())
    
df = df.drop(['user_tags'], axis=1)

In [138]:
bool_col = ['creative_is_jump','creative_is_download','creative_is_js','creative_is_voicead','creative_has_deeplink','app_paid']
for i in bool_col:
    df[i] = df[i].astype(int)

In [139]:
def encode_feature(values):
    uniq = values.unique()
    mapping = dict(zip(uniq,range(1,len(uniq) + 1)))
    return values.map(mapping)
                      
zip_col = ['city','province','carrier','devtype','make','model','nnt','os','osv','os_name','adid','advert_id','orderid','campaign_id','creative_id','creative_tp_dnf','app_cate_id','app_id','inner_slot_id','creative_type','advert_name','advert_industry_inner_split_0','advert_industry_inner_split_1']
for i in tqdm_notebook(zip_col):
    df[i] = encode_feature(df[i])

In [140]:
df.head()

,instance_id,time,city,province,carrier,devtype,make,model,nnt,os,osv,os_name,adid,advert_id,orderid,campaign_id,creative_id,creative_tp_dnf,app_cate_id,app_id,inner_slot_id,creative_type,creative_width,creative_height,creative_is_jump,creative_is_download,creative_is_js,creative_is_voicead,creative_has_deeplink,app_paid,advert_name,click,advert_industry_inner_split_0,advert_industry_inner_split_1,user_tags_0,user_tags_1,user_tags_2,user_tags_3,user_tags_4,user_tags_5,user_tags_6,user_tags_7,user_tags_8,user_tags_9,user_tags_10,user_tags_11,user_tags_12,user_tags_13,user_tags_14,user_tags_15,user_tags_16,user_tags_17,user_tags_18,user_tags_19,user_tags_20,user_tags_21,user_tags_22,user_tags_23,user_tags_24,user_tags_25,user_tags_26,user_tags_27,user_tags_28,user_tags_29,user_tags_30,user_tags_31,user_tags_32,user_tags_33,user_tags_34,user_tags_35,user_tags_36,user_tags_37
0,86294719979897807,2190219034,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1280,720,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2699289844928136052,2190221070,2,2,2,1,2,2,1,1,2,1,2,1,2,1,2,2,2,2,2,1,960,640,1,0,0,0,0,0,1,0,1,1,1,1,1,0,1,0,1,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,3117527168445845752,2190219793,3,3,2,1,3,3,1,1,3,1,3,2,3,2,3,3,3,3,3,1,960,640,1,0,0,0,0,0,2,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3398484891050993371,2190221704,4,1,3,1,4,4,1,1,4,1,4,3,4,3,4,4,3,4,4,2,1280,720,1,0,0,0,0,0,3,0,3,3,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,2035477570591176488,2190220024,5,1,1,1,5,5,2,2,5,2,5,1,5,1,5,2,4,5,5,1,960,640,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [141]:
df.to_csv('./data/df_preprocess.csv', index=None)